In [1]:
import pandas as pd
import re
import string
import unidecode
from collections import Counter
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer


with open('Ana.txt',encoding="UTF-8") as f:
    article_text = f.read().replace("\n"," ")

https://www.congreso.es/busqueda-de-intervenciones

In [2]:
def discurso(cont):
    discurso = ""
    cont_func = cont
    
    for speech in palabras[cont:]:
        
        if "señor" in speech and palabras[cont_func+1].isupper():#Si encuentra nombre para el bucle
                    
            break

            
        discurso += speech + " "
        cont_func += 1
        
        
        
    return discurso

In [3]:
cont = 0
palabras = article_text.split()
lista = []#Creara el dataframe despues
disc = 1 #Estrablece el orden de los discursos
for i in palabras:
    nombre = ""
    int_cont = 1

    if "señor" in i and palabras[cont+1].isupper():#Check if name
        
        for subloop in palabras[cont+1:]:#Sacamos el nombre
            
            if subloop.isupper() == False:
                
                lista.append([nombre,discurso(int_cont+cont),disc])
                disc +=1
                
                break
            nombre += subloop + " " #Añadimos el nombre del interventor
        
        
            int_cont += 1   
        
        
   
        
    cont += 1

In [4]:
data = pd.DataFrame(lista,columns = ["Interventor","Discurso","Orden"])
data

,Interventor,Discurso,Orden
0,PRESIDENTA:,"Buenos días, señorías. Se reanuda la sesión. S...",1
1,SÁNCHEZ PÉREZ-CASTEJÓN,(candidato a la Presidencia del Gobierno): Gra...,2
2,PRESIDENTA:,"Señorías, por favor. (Rumores). Ruego silencio...",3
3,SÁNCHEZ PÉREZ-CASTEJÓN,(candidato a la Presidencia del Gobierno): Han...,4
4,PRESIDENTA:,Ruego silencio. (El señor García Egea pronunci...,5
...,...,...,...
88,PRESIDENTA:,"Gracias, señor García. En último lugar, por el...",89
89,MARTÍNEZ OBLANCA:,"Señora presidenta, señorías, señor candidato, ...",90
90,PRESIDENTA:,"Gracias, señor Martínez Oblanca. Por el Grupo ...",91
91,LASTRA FERNÁNDEZ:,"Presidenta, señorías, el pasado domingo comenz...",92


# Analisis 

In [86]:
def palabras_repetidas(df,nombre,columna,comparar = False):
    df['temp_list'] = df[columna].apply(lambda x:str(stopwords_rem(x)).split())
    l=[]
    top = Counter([item for sublist in df['temp_list'] for item in sublist if len(item)>3 and item != "luis" and item != "enrique"])
    
    temp = pd.DataFrame(top.most_common(20))
    t2=pd.DataFrame(top.most_common(400000))
    t2.columns = ['Palabras ','Cuenta']
    total=t2.sum()[1]
    t2["Porcentaje"]=round(t2["Cuenta"]/total*100,4)
    temp.columns = ['Palabras ','Cuenta']
    temp["Porcentaje"]=round(t2["Cuenta"]/total*100,4)
    
    if comparar == True:
        
        for i,row in temp.iterrows():
            temp.at[i,"TOT"] = df_palabras_totales[df_palabras_totales == temp.at[i,"Palabras "]].iloc[0][1]
        
    return temp.style.background_gradient(cmap='Blues'),t2

def stopwords_rem(text):
    final_x=""
    text=limpia(text)
    #text=" ".join(tokenize(text))
    for i in text.split():
        if i not in stopwords_es:
            final_x+=i+" "
    return final_x
     
def limpia(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = unidecode.unidecode(text)
    
    

    return text

stopwords_es=stopwords.words("spanish")
stopwords_es.append("par")


In [6]:
data.Interventor.unique()

array(['PRESIDENTA: ', 'SÁNCHEZ PÉREZ-CASTEJÓN ', 'CASADO BLANCO: ',
       'ABASCAL CONDE: ', 'IGLESIAS TURRIÓN: ', 'BORRÀS CASTANYER: ',
       'ERREJÓN GALVÁN: ', 'BALDOVÍ RODA: ', 'ORAMAS GONZÁLEZ-MORO: ',
       'QUEVEDO ITURBE: ', 'REGO CANDAMIL: ', 'MAZÓN RAMOS: ',
       'GUITARTE GIMENO: ', 'BASSA COLL: ', 'ARRIMADAS GARCÍA: ',
       'ESTEBAN BRAVO: ', 'MATUTE GARCÍA DE JALÓN: ', 'VEHÍ CANTENYS: ',
       'GARCÍA ADANERO: ', 'MARTÍNEZ OBLANCA: ', 'LASTRA FERNÁNDEZ: '],
      dtype=object)

In [73]:
display(palabras_repetidas(data[data.Interventor.str.contains("SÁNCHEZ PÉREZ-CASTEJÓN")],"g","Discurso")[0])

C:\Users\melfalso\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Palabras g,Cuenta,Porcentaje
0,senorias,21,2.089600
1,gobierno,17,1.691500
2,tambien,13,1.293500
3,pais,10,0.995000
4,espana,9,0.895500
5,coalicion,9,0.895500
6,elecciones,8,0.796000
7,derechos,8,0.796000
8,bloqueo,7,0.696500
9,espanoles,7,0.696500


In [8]:
display(palabras_repetidas(data[data.Interventor.str.contains("CASADO BLANCO: ")],"g","Discurso")[0])

C:\Users\melfalso\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Palabras g,Cuenta,Porcentaje
0,espana,16,2.043400
1,usted,11,1.404900
2,sanchez,9,1.149400
3,libertad,9,1.149400
4,gobierno,8,1.021700
5,sino,8,1.021700
6,historia,7,0.894000
7,senor,7,0.894000
8,espanoles,7,0.894000
9,democracia,6,0.766300


# Sentimiento

In [9]:
def analizador(text):
    b =limpia(text) #
    #b = str(TextBlob(text).translate(to="en"))
    sid=SentimentIntensityAnalyzer()
    return sid.polarity_scores(b),b


In [10]:
data["Sent"]=""
for i,row in data.iterrows():
    
    if i>-1: 
        #print(i)
        #try:
            data.at[i,"Sent"]=analizador(row["Discurso"])[0]["compound"]
            data.at[i,"Aplauso"]=row["Discurso"].count("plauso")
            
            #sleep(1)
        #except: print("Error")

In [12]:
data = data[~(data.Interventor=="PRESIDENTA: ")]


In [38]:
#Eliminamos intervenciones residuales de menos de 200 palabras
data = data[data["Discurso"].apply(lambda x: len(x.split(' ')) > 200)] #https://stackoverflow.com/questions/42895061/how-to-remove-a-row-from-pandas-dataframe-based-on-the-length-of-the-column-valu

In [39]:
data.sort_values("Aplauso",ascending=False)

,Interventor,Discurso,Orden,Sent,Aplauso
91,LASTRA FERNÁNDEZ:,"Presidenta, señorías, el pasado domingo comenz...",92,-0.875,13.0
23,IGLESIAS TURRIÓN:,"Muchas gracias, señora presidenta. Muchas grac...",24,-0.9468,12.0
19,ABASCAL CONDE:,"Gracias, señora presidenta. Teniendo en cuenta...",20,-0.9252,9.0
11,CASADO BLANCO:,"Gracias, señora presidenta. Quiero comenzar mi...",12,-0.9875,9.0
7,SÁNCHEZ PÉREZ-CASTEJÓN,(candidato a la Presidencia del Gobierno): El ...,8,-0.9788,6.0
75,MATUTE GARCÍA DE JALÓN:,"Presidenta, integrantes de la Mesa, también de...",76,-0.9638,5.0
1,SÁNCHEZ PÉREZ-CASTEJÓN,(candidato a la Presidencia del Gobierno): Gra...,2,-0.9922,5.0
87,GARCÍA ADANERO:,"Muchas gracias, señora presidenta. Señor Sánch...",88,-0.977,4.0
31,BALDOVÍ RODA:,"Señor Abascal, perdone que insista, a la vista...",32,-0.7805,4.0
21,ABASCAL CONDE:,"Concluyo inmediatamente, presidenta. Vimos aye...",22,-0.9286,3.0


In [40]:
data.sort_values("Sent",ascending=False)

,Interventor,Discurso,Orden,Sent,Aplauso
37,REGO CANDAMIL:,"Gracias, señora presidenta. Galiza deixará de ...",38,0.3182,1.0
45,BASSA COLL:,Buenos días. Antes de nada y de parte de Esque...,46,0.1027,3.0
77,MATUTE GARCÍA DE JALÓN:,Estamos aquí para defender nuestro derecho a c...,78,-0.126,1.0
35,QUEVEDO ITURBE:,"Gracias, señora presidenta. Primera considerac...",36,-0.4215,2.0
79,MATUTE GARCÍA DE JALÓN:,"En 1978 desde esta misma tribuna, Francisco Le...",80,-0.5859,0.0
69,ARRIMADAS GARCÍA:,"... necesita extremismo -acabo, señora preside...",70,-0.6808,0.0
9,SÁNCHEZ PÉREZ-CASTEJÓN,(candidato a la Presidencia del Gobierno): Señ...,10,-0.7003,3.0
31,BALDOVÍ RODA:,"Señor Abascal, perdone que insista, a la vista...",32,-0.7805,4.0
67,ARRIMADAS GARCÍA:,Ustedes han presionado a quienes han querido p...,68,-0.8205,2.0
63,ARRIMADAS GARCÍA:,"Gracias, señora presidenta. En primer lugar qu...",64,-0.8706,3.0


In [41]:
data.to_excel("Congreso.xlsx")

# % Repeticion de palabras

In [45]:
cadena = ""
for i,row in data.iterrows(): cadena +=row.Discurso # Se juntan todos los discursos

In [50]:
listaPalabras = stopwords_rem(cadena).split()

frecuenciaPalab = []
for w in listaPalabras:
    frecuenciaPalab.append(listaPalabras.count(w))


palabras = list(zip(listaPalabras, frecuenciaPalab))

In [87]:
df_palabras_totales = pd.DataFrame(palabras).sort_values(1,ascending=False).drop_duplicates()

In [94]:
df_palabras_totales

,0,1
258,gobierno,81
1526,mas,59
4269,espana,55
4215,senor,52
5274,hoy,51
...,...,...
3212,explicar,1
3213,mala,1
3215,intolerancia,1
3217,video,1
